Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from scipy.stats import randint
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.decomposition import TruncatedSVD

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

import spacy
nlp = spacy.load("en_core_web_lg")

import pandas as pd
import numpy as np
import re

#other imports for testing
import unidecode
from word2number import w2n
import gensim.downloader as api

#ntlk imports
import nltk
import nltk.corpus

#tokenizing
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

#stemming
from nltk.stem import PorterStemmer

#lemmitizing
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

#modeling
import lightgbm as lgb
from lightgbm import LGBMClassifier


/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping, defaultdict
/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_lg' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and availab

In [2]:
# Bringing in the data
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
print(train.shape, test.shape)

(4087, 3) (1022, 2)


In [3]:
pd.set_option('display.max_colwidth', 0)
train['description'].sample(2)

1747    \nLight straw. Initially this is quite hot and a little dumb, with whiffs of Indian spice — think turmeric and curry leaf — along with mint sauce (but no lamb) and a tickle of peat. The palate is quite intense and hot, with powdered almond, a grassy edge, and concentrated sweetness that starts in the center and builds toward the back palate. Subtle, but can’t help wishing there was just a little more say from the cask. £59                                                                                      
2905    \nWillingly, we travel further along the Big Dipper with whisky maker Henric Molin. His latest creation exudes rich, dark, chewy toffee, Brazil nut shells, chopped dates, licorice, stone fruits, grilled beef mushrooms, and worked leather. The flavor starts slowly, like trying to ignite a fire of damp twigs. Once established, there is good weight and density, with treacle, dried fig, black cherry, and cinnamon, but the finish diminishes quickly after swallowing, leavi

In [4]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts(normalize=True)

1    0.704918
0    0.279178
2    0.015904
Name: ratingCategory, dtype: float64

In [5]:
# Read a few reviews from the "Excellent" category
pd.set_option('display.max_colwidth', -1)
train[train.ratingCategory == 0].head(1)

<ipython-input-5-cf08eddbe278>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,id,description,ratingCategory
1,3861,"\nAn uncommon exclusive bottling of a 6 year old cask strength malt. Light gold in color, the nose is vegetal, more peat bog than peat smoke, with an undercurrent of pastry cream and rose. It’s an odd combination of aromas. The entry is flavorful and inviting with smoked pineapple, clove, and rose. Peak smoke arrives in full force in the mid-palate, which drops the sweet and becomes spicy. The finish is mostly smoke, but with a pleasant minty coolness. (Wyoming only)",0


In [6]:
# Read a few reviews from the "Poor" category
train[train.ratingCategory == 2].sample(1)

,id,description,ratingCategory
3504,5078,"\nIts best attributes are vanilla, toasted coconut, and tropical fruit, but the rest of this grain whisky is a bit of a disappointment. It is thin, and at times harsh. A paint thinner component is evident (especially on the nose), along with more wood on the finish than this thin body can handle. Two Carsebridge grain Scotch whiskies from Duncan Taylor which I have tasted recently were much better: richer and creamier, and with more balance.",2


### Split the Training Set into Train/Validation

In [7]:
X_train, X_val, y_train, y_val = train_test_split(train['description'], 
                                                    train['ratingCategory'], 
                                                    test_size=0.5, 
                                                    stratify=train['ratingCategory'],
                                                    random_state=42)

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(2043,) (2044,) (2043,) (2044,)


In [9]:
X_val.head(1)

2255    \nThe tenth limited release of Grandeur was matured in a mix of Pedro Ximénez and oloroso sherry casks. It offers a nose of ginger, raisins, old leather, smoky orange, blackcurrants, and walnuts. The palate is voluptuous, with full sherry notes, figs, dates, cinnamon, and cloves. Licorice and spicy dark chocolate in the lingering finish. (240 bottles for the U.S.)
Name: description, dtype: object

In [10]:
def clean_text(X):
    
    X = X.apply(lambda x: x.lower())
    X = X.apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    X = X.apply(lambda elem: re.sub(r"\d+", "", elem))
    
    return X

In [11]:
X_train = clean_text(X_train)
X_val = clean_text(X_val)
X_val.head(1)

2255    the tenth limited release of grandeur was matured in a mix of pedro ximnez and oloroso sherry casks it offers a nose of ginger raisins old leather smoky orange blackcurrants and walnuts the palate is voluptuous with full sherry notes figs dates cinnamon and cloves licorice and spicy dark chocolate in the lingering finish  bottles for the us
Name: description, dtype: object

In [12]:
#stopwords
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/codyreece/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
#function to remove stopwords
def no_stop(X):

    X = X.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    return X

X_train = no_stop(X_train)
X_val = no_stop(X_val)
X_val.head(1) 

2255    tenth limited release grandeur matured mix pedro ximnez oloroso sherry casks offers nose ginger raisins old leather smoky orange blackcurrants walnuts palate voluptuous full sherry notes figs dates cinnamon cloves licorice spicy dark chocolate lingering finish bottles us
Name: description, dtype: object

In [14]:
#tokenizing
# import nltk 
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

def tokeitup(X):
    X = X.apply(lambda x: word_tokenize(x))

    return X

#applying tokenizing function
X_train = tokeitup(X_train)
X_val = tokeitup(X_val)
X_val.head(1)

2255    [tenth, limited, release, grandeur, matured, mix, pedro, ximnez, oloroso, sherry, casks, offers, nose, ginger, raisins, old, leather, smoky, orange, blackcurrants, walnuts, palate, voluptuous, full, sherry, notes, figs, dates, cinnamon, cloves, licorice, spicy, dark, chocolate, lingering, finish, bottles, us]
Name: description, dtype: object

In [15]:
# from nltk.stem import PorterStemmer 
# def word_stemmer(text):
#     stem_text = [PorterStemmer().stem(i) for i in text]
#     return stem_text
# X_train = X_train.apply(lambda x: word_stemmer(x))
#X_train.head(1)

In [16]:
#Lemmatizing
# nltk.download('wordnet')
# from nltk.stem import WordNetLemmatizer
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text
X_train = X_train.apply(lambda x: word_lemmatizer(x))
X_val = X_val.apply(lambda x: word_lemmatizer(x))
X_val.head(1)

2255    [tenth, limited, release, grandeur, matured, mix, pedro, ximnez, oloroso, sherry, cask, offer, nose, ginger, raisin, old, leather, smoky, orange, blackcurrants, walnut, palate, voluptuous, full, sherry, note, fig, date, cinnamon, clove, licorice, spicy, dark, chocolate, lingering, finish, bottle, u]
Name: description, dtype: object

In [17]:
# #tagging
# def word_pos_tagger(text):
#     pos_tagged_text = nltk.pos_tag(text)
#     return pos_tagged_text

# nltk.download('averaged_perceptron_tagger')

# X_train = X_train.apply(lambda x: word_pos_tagger(x))
# X_val = X_val.apply(lambda x: word_pos_tagger(x))
# X_val.head(1)

In [18]:
# #chunking
# import spacy

# nlp = spacy.load("en_core_web_lg")
# doc = nlp(str(X_train[:1]))

# for chunk in doc.noun_chunks:
#     print(chunk.text, chunk.root.text, chunk.root.dep_,
#             chunk.root.head.text)

In [19]:
print("Changes to X_train:")
print(X_train.head(1))
print("____")

print("Data type for X_train: ")
print(type(X_train))
print("____")
print(X_train.shape)
print("_________________")
print("_________________")
print("Changes to X_val:")
print("_____")
print(X_val.head(1))
print("_____")
print("Data type for X_val: ")
print(type(X_val))
print("_____")
print(X_val.shape)


Changes to X_train:
3923    [highrye, bourbon, mashbill, small, batch, whiskey, annual, limited, release, nose, offer, inviting, medley, chocolatecovered, cherry, pound, cake, buttered, toast, blackberry, licorice, palate, mellow, honeyed, note, cedar, chocolate, chip, cooky, blackberry, jam, coconut, shaving, cigar, wrapper, completed, finish, brown, sugar, chocolate, peanut, shell, cracked, pepper]
Name: description, dtype: object
____
Data type for X_train: 
<class 'pandas.core.series.Series'>
____
(2043,)
_________________
_________________
Changes to X_val:
_____
2255    [tenth, limited, release, grandeur, matured, mix, pedro, ximnez, oloroso, sherry, cask, offer, nose, ginger, raisin, old, leather, smoky, orange, blackcurrants, walnut, palate, voluptuous, full, sherry, note, fig, date, cinnamon, clove, licorice, spicy, dark, chocolate, lingering, finish, bottle, u]
Name: description, dtype: object
_____
Data type for X_val: 
<class 'pandas.core.series.Series'>
_____
(2044,)


In [20]:
X_train_join = X_train.apply(lambda x: " ".join(x))
X_val_join = X_val.apply(lambda x: " ".join(x))

In [21]:
print(train['description'].shape, X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(4087,) (2043,) (2044,) (2043,) (2044,)


In [22]:
X_val.head(2)

2255    [tenth, limited, release, grandeur, matured, mix, pedro, ximnez, oloroso, sherry, cask, offer, nose, ginger, raisin, old, leather, smoky, orange, blackcurrants, walnut, palate, voluptuous, full, sherry, note, fig, date, cinnamon, clove, licorice, spicy, dark, chocolate, lingering, finish, bottle, u]                                                                            
3847    [little, beauty, maple, syrup, pecan, sliced, peach, vanillaladen, breadandbutter, pudding, soft, bakedapple, tart, smooth, sticky, toffee, pudding, red, apple, oozing, caramel, fine, layer, spice, mouthfeel, silky, rounded, effortlessly, elegant, finish, walnut, clove, marron, glac, douglas, laing, brought, amazing, grain, lately, better, ever, kl, wine, exclusive, bottle]
Name: description, dtype: object

### Define Pipeline Components

In [ ]:
#Checking for the best number of workers
N_JOBS = [None, 1, 2, 3, 4, -1]
import time
for n_jobs in N_JOBS:
        start = time.perf_counter()
        print('n_jobs = {}, perf_counter = {}'.format(n_jobs, time.perf_counter() - start))

In [ ]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()
lbm = LGBMClassifier(random_state=42, objective='binary', reg_lambda=0.2)
xgb = XGBClassifier(objective = 'multi:softmax', num_class = 6,
                                       booster='gbtree',eta =.1,  max_depth = 12,
                                       subsample=.8, n_estimators=120, random_state = 42,
                                       eval_metric = 'merror', colsample_bytree = 1,
                                       tree_method = 'exact', maximize=False)
vect2 = CountVectorizer()
svd = TruncatedSVD()

mnb = MultinomialNB()
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect2), 
#                  ('svd', svd),
                 # Classifier
                 ('clf', lbm)
                ])

In [ ]:
%%time
parameters = {
#     'vect__max_df': np.linspace(0.1, 1, 10),
#     'vect__min_df': (.01, .02),
#     'vect__max_features': (2500, 5000, 10000, 12000),
#     'vect__norm': [None, 'l1', 'l2'],
#     'vect__binary': [True, False],
#     'clf__alpha': np.linspace(0.5, 1.5, 6),
#     'clf__fit_prior': [True, False]
    
    'clf__n_estimators':( 10, 25, 50, 100),
    'clf__max_depth':(6, 7, 8, 9, 12, 15),
    'clf__boosting_type':('gbdt', 'dart'),
    'clf__num_leaves': (75, 100),
    'clf__learning_rate': (.2, .3, .4), 
    'clf__feature_fraction': (.5, .75, .9)
}

grid_search = RandomizedSearchCV(pipe,parameters, cv=32, n_jobs=-1, verbose=10) #GridSearchCV RandomizedSearchCV
grid_search.fit(X_train_join, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
subNumber = 6

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
vect2 = CountVectorizer()
svd = TruncatedSVD(n_components=100,
                   random_state=42,# Just here for demo. 
                   algorithm='randomized')
lbm = LGBMClassifier(random_state=42, objective='binary', reg_lambda=0.2)

pipe = Pipeline([
    ('vect', vect2),      # TF-IDF Vectorizer
    ('svd', svd),        # Truncated SVD Dimensionality Reduction
    ('clf', mnb)         # RandomForest Classifier
])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
parameters = {
    'vect__max_df': np.linspace(0.1, 1, 10),
    'vect__min_df': (.01, .02),
    'vect__max_features': (2500, 5000, 10000, 12000, 15000, 99999),
    'vect__norm': [None, 'l1', 'l2'],
    'vect__binary': [True, False],
    'clf__alpha': np.linspace(0.5, 1.5, 6),
    'clf__fit_prior': [True, False]
#     'vect__max_df': (.3, .4, .5),
#     'vect__min_df': (.005, .01, .02),
#     'vect__max_features': (1200, 2500, 5000, 10000, 12000),
#     'clf__n_estimators':( 10, 25, 50, 100),
#     'clf__max_depth':(6, 7, 8, 9, 12, 15),
#     'clf__boosting_type':('gbdt', 'dart'),
#     'clf__num_leaves': (50, 75, 100),
#     'clf__learning_rate': (.2, .3, .4), 
#     'clf__feature_fraction': (.5, .75, .9)
}

grid_search = RandomizedSearchCV(pipe,parameters, cv=50, n_jobs=-1, verbose=10)
grid_search.fit(X_train_join, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt7/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [ ]:
# Apply to your Dataset
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint

param_dist = {
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./whiskey-reviews-dspt7/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?